In [2]:
from Bio import SeqIO
from Bio.Seq import Seq

## Lector de archivos FASTA

Para empezar con la práctica, se crea un código que sea capaz de leer archivos en formato FASTA. El archivo elegido es 'gene.fna', que contiene secuencias de nucleótidos del gen TP53, un gen muy estudiado ya que codifica la proteína p53, clave en la regulación del ciclo celular asociada a cáncer cuando presenta mutaciones.

In [5]:
archivo = "data/gene.fna"

secuencias = []

for record in SeqIO.parse(archivo, "fasta"):
    secuencias.append(record)

print(f"Secuencias leídas: {len(secuencias)}")
print("Primera secuencia: ", secuencias[0].seq[:50])
print("ID:", secuencias[0].id)
print("Longitud:", len(secuencias[0].seq))

Secuencias leídas: 2
Primera secuencia:  CTCAAAAGTCTAGAGCCACCGTCCAGGGAGCAGGTAGCTGCTGGGCTCCG
ID: NC_000017.11:c7687490-7668421
Longitud: 19070


Como se observa en los resultados, el archivo contiene dos secuencias, pero nos centraremos solo en la primera.
 

## Transcripción

Ahora, se crea la secuencia de ARNm de la primera secuencia de ADN mediante el proceso de *transcipción*. Para ello, se usa la función `transcribe` de `Biopython`, que asume la secuencia como si fuese la hebra codificante (5'→ 3') y simplemente convierte las T en U:

In [6]:
dna_seq = secuencias[0].seq
mRNA_seq = dna_seq.transcribe()

print("ADN:", dna_seq[:50])
print("ARNm:", mRNA_seq[:50])

ADN: CTCAAAAGTCTAGAGCCACCGTCCAGGGAGCAGGTAGCTGCTGGGCTCCG
ARNm: CUCAAAAGUCUAGAGCCACCGUCCAGGGAGCAGGUAGCUGCUGGGCUCCG


## Transcripción con secuencia de clase

Para este ejercicio, como se proporciona tanto la hebra molde como la codificante, se hará el código para ambas.

### Código para hebra codificante

In [7]:
cod = Seq("ATGCCTGAATGC")
mRNA_seq = cod.transcribe()

print("Hebra codificante: ", cod)
print("ARNm:", mRNA_seq)

Hebra codificante:  ATGCCTGAATGC
ARNm: AUGCCUGAAUGC


### Codigo para hebra molde

In [12]:
molde = Seq("TACGGACTTACG")   
molde_5to3 = molde[::-1]
codificante = molde_5to3.reverse_complement()  
arnm = codificante.transcribe()   

print("Cadena molde (3'->5'):", molde)
print("Cadena molde (5'->3'):", molde_5to3)
print("Hebra codificante (5'->3'):", codificante)
print("ARNm (5'->3'):", arnm)


Cadena molde (3'->5'): TACGGACTTACG
Cadena molde (5'->3'): GCATTCAGGCAT
Hebra codificante (5'->3'): ATGCCTGAATGC
ARNm (5'->3'): AUGCCUGAAUGC


Ya que `Biopython` no genera el ARNm a partir de la cadena molde, si se quisiera hacer así, primero hay que obtener la cadena complementaria.

La cadena molde va en dirección 3' → 5', pero Biopython asume que la cadena que se le pasa siempre está en 5'→3', por lo que primero se debe cambiar la hebra molde de sentido.

Luego, se obtiene la hebra complementaria a la molde mediante la función `complement` de `Biopython`. El resultado obtenido es la hebra codificante.

Finalmente, se hace lo mismo que se hizo en el primer ejemplo (usar la función `transcribe`), obteniendo así la secuencia de ARNm,

Comos se puede observar en los resultados obtenidos, la hebra de ARNm es la misma que se obtuvo en el ejercicio "a mano".

## Experimentos

Una ejercicio muy interesante es observar cómo cambia el resultado dependiendo de qué hebra se usa como punto de partida.

Cuando se habla de replicación y transcripción, como se dijo antes, hay que tener muy clara la orientación:
- La cadena molde (3'→5') sirve de plantilla para sintetizar el ARNm en dirección 5'→3'
- La cadena codificante (5'→3') tiene la misma secuencia que el ARNm, pero con T en vez de U

Si se invierte la hebra, o se toma la complementaria de la complementaria, se puede perder la orientación biológica correcta.

A continuación, se lleva a cabo una serie de experimentos para comprobar todo esto:

#### Transcripción directa del FASTA

En este caso, se está asumiendo que el archivo FASTA trae la *hebra codificante en orientación 5'→3'*

In [17]:
mRNA_seq = dna_seq[:50].transcribe()
print("Original (asumida codificante):", mRNA_seq)

Original (asumida codificante): CUCAAAAGUCUAGAGCCACCGUCCAGGGAGCAGGUAGCUGCUGGGCUCCG


#### Usar la complementaria

En este caso, se está construyendo la hebra complementaria de la que se tiene en el FASTA, por lo que se estaría obteniendo algo parecido a la cadena molde, pero no invertida.

Si el FASTA contenía la molde, ahora se tendría algo parecido a la codificante, pero también mal orientada.

El resultado de ARNm no tiene sentido biológico, porque el proceso real no funciona tomando la complementaria "cruda". El ARN polimerasa necesita leer la cadena molde en orientación 3'→5', por eso la hebra obtenida no tendría sentido.

In [8]:
mRNA_seq_from_complement = dna_seq.complement().transcribe()
print("Complementaria:", mRNA_seq_from_complement[:50])


Complementaria: GAGUUUUCAGAUCUCGGUGGCAGGUCCCUCGUCCAUCGACGACCCGAGGC


#### Usar la reversa complementaria

In [9]:
mRNA_seq_from_reverse = dna_seq.reverse_complement().transcribe()

print("Reversa complementaria (molde):", mRNA_seq_from_reverse[:50])


Reversa complementaria (molde): UGGCAGCAAAGUUUUAUUGUAAAAUAAGAGAUCGAUAUAAAAAUGGGAUA


Lo que está pasando aquí es que, cuando se utiliza la función `reverse_complement()` en Biopython, se obtiene la hebra complementaria pero invertida, es decir, en orientación 3'→5'. Esto simula la hebra molde real que emplea la ARN polimerasa durante la transcripción.

Al aplicar después `.transcribe()`, se genera el ARNm en dirección 5'→3', que corresponde a la secuencia biológicamente correcta. De este modo, si el archivo FASTA contiene la hebra codificante, obtener su reversa complementaria equivale a recuperar la hebra molde y, al transcribirla, se produce el ARNm tal como ocurre en la célula.